In [1]:
from rutas import data_clean
maestro = data_clean()
maestro = maestro[(maestro["min"].apply(lambda x: x[:4])=="2022")|(maestro["min"].apply(lambda x: x[:4])=="2021")|(maestro["min"].apply(lambda x: x[:4])=="2019")]

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
import warnings

warnings.filterwarnings("ignore")

y = np.array(maestro["calif"])
X = maestro.copy()
X.drop(["calificacion", "calif"], axis=1, inplace=True)

X = X[["COLOR", "ESTETICA", "LARGO", "SILUETA", "BASE TELA"]]

categorical_columns = maestro.select_dtypes(include='object').columns
df_copy = maestro.copy()
label_encoders = {col: LabelEncoder() for col in categorical_columns}
for col in categorical_columns:
    df_copy[col] = label_encoders[col].fit_transform(df_copy[col])
features = df_copy[["COLOR", "ESTETICA", "LARGO", "SILUETA", "BASE TELA"]]
target = df_copy['calif']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)


<h2 style="text-align:center;">Regresion lineal multivariada</h2>

In [126]:
dict_criterios = {"HOMBRE":
    {"BUZO":["COLOR", "ESTIICA", "TIPO"], #TIPO PULLOVER - HOODIE
    "CAMISA": ["ESTETICA", "FIT", "COLOR"],
    "CAMISETA":["COLOR", "ESTETICA", "FIT", "TIPO"], #TIPO MANGA CORGA - MANGA LARGA
    "CHAQUETA": ["COLOR", "ESTETICA"],
    "JEAN": ["COLOR", "ESTETICA"],
    "JOGGER": ["COLOR", "ESTETICA"],
    "SHORTS": ["COLOR", "ESTETICA"],
    "SHORTS PLANO": ["COLOR", "ESTETICA"],
    "PANTALON": ["COLOR"],
    "POLO": ["COLOR", "TEJIDO", "PUNTO CORAZON"],
    "TANK": ["COLOR", "ESTETICA", "TIPO"] #TIPO CHOMPA
    },
    "MUJER":{"BUZO":["COLOR", "ESTIICA", "TIPO", "LARGO", "BASE TELA", "SILUETA"], #TIPO PULLOVER - HOODIE ## VALIDAR SI TENEMOS SOLUETA
    "CAMISA": ["ESTETICA", "TIPO", "COLOR", "LARGO"], #TIPO MANGA CORGA - MANGA LARGA
    "CAMISETA":["COLOR", "ESTETICA", "LARGO", "SILUETA"], #TIPO MANGA CORGA - MANGA LARGA
    "CHAQUETA": ["COLOR", "ESTETICA", "TIPO", "LARGO", "BASE TELA", "SILUETA"],
    "FALDA": ["ESTETICA", "BASE TELA", "LARGO"],
    "HAREM" : ["SILUETA", "MOLDE", "COLOR", "ESTETICA", "BASE TELA"],
    "JEAN": ["COLOR", "ESTETICA", "TIPO", "TIRO"], #TIRO EN LA SUBCATEGORIA
    "JOGGER": ["COLOR", "ESTETICA", "SILUETA", "BASE TELA", "TIPO"],
    "PANTALON": ["COLOR", "ESTETICA", "TIRO", "TIPO"], #TIRO EN LA SUBCATEGORIA
    "R DEPORTIVA": ["COLOR", "ESTETICA", "BASE TELA", "SILUETA"],
    "SHORTS": ["COLOR", "ESTETICA", "SILUETA", "BASE TELA"],
    "SHORTS DENIM": ["COLOR", "ESTETICA", "BASE TELA", "TIRO"],
    "TOP": ["COLOR", "LARGO", "SILUETA", "ESTETICA", "TIPO"],
    "TOP ML": ["COLOR", "LARGO", "SILUETA", "ESTETICA", "TIPO"],
    "VESTIDO": ["BASE TELA", "SILUETA", "LARGO", "ESTETICA", "COLOR", "TIPO"], 
    "VESTIDO DE BAÑO": ["COLOR", "ESTETICA", "MOLDE"]
    }
}

<h4 style="text-align:center;">Modelo</h4>

In [41]:
for i in range(1000):
    try:
        (X_train, X_test, y_train, y_test) = train_test_split(
            X,
            y,
            train_size = 0.7)
        pipeline = Pipeline(
                steps=[
                    (
                        "column_transformer",
                        make_column_transformer(
                            (
                                OneHotEncoder(),
                                make_column_selector(dtype_include=object)
                            ), 
                            remainder = "passthrough"
                        ),
                    ),
                    (
                        "selectKBest",
                        SelectKBest(score_func=f_regression),
                    ),
                    (
                        "linearRegression",
                        LinearRegression(),
                    ),
                ],
            )
        param_grid = {
                "selectKBest__k": range(1, len(X.columns)),
            }
        gridSearchCV = GridSearchCV(
                estimator=pipeline,
                param_grid=param_grid,
                scoring="neg_mean_squared_error",
                refit=True,
                return_train_score=False,
            )
        gridSearchCV.fit(X_train, y_train)
        y_train_pred = gridSearchCV.predict(X_train)
        y_test_pred = gridSearchCV.predict(X_test)
        mse_train = mean_squared_error(
            y_train_pred,
            y_train,
        ).round(2)
        mse_test = mean_squared_error(
            y_test_pred,
            y_test,
        ).round(2)

        r_2 = r2_score(y_pred = y_test_pred, y_true = y_test)

        print(f"mse train: {mse_train} \nmse tres: {mse_test} \nEl R2 es {r_2}")
        break
    except: continue 
print(i)

mse train: 1.5 
mse tres: 1.73 
El R2 es -0.07744033056851274
2


<h2 style="text-align:center;">Red Neuronal TensorFlow</h2>

In [158]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Softmax


y = np.array(maestro["calif"])
X = maestro.copy()
X.drop(["calificacion", "calif"], axis=1, inplace=True)


X = X[["COLOR", "ESTETICA", "LARGO", "SILUETA"]]
X = X.to_numpy()

training_data = make_column_transformer(
                            (
                                OneHotEncoder(),
                                make_column_selector(dtype_include=object)
                            ), 
                            remainder = "passthrough"
                        )
training_data = np.array(training_data)
target_data = y

# Seleccionar las columnas del dataframe que son categóricas
categorical_columns = maestro.select_dtypes(include='object').columns

# Crear una copia del dataframe para evitar modificar el original
df_copy = maestro.copy()

# Crear una instancia de LabelEncoder para cada columna categórica
label_encoders = {col: LabelEncoder() for col in categorical_columns}

# Aplicar el LabelEncoder a cada columna categórica del dataframe
for col in categorical_columns:
    df_copy[col] = label_encoders[col].fit_transform(df_copy[col])

# Seleccionar las columnas del dataframe que se utilizarán como características
features = df_copy[["COLOR", "ESTETICA", "LARGO", "SILUETA"]]

# Seleccionar la columna del dataframe que se utilizará como objetivo
target = df_copy['calif']

# Crear un modelo de redes neuronales con Keras
model = Sequential()

# Añadir una capa de entrada con la misma cantidad de neuronas que de características
model.add(Dense(features.shape[1], input_dim=features.shape[1], activation='relu'))

# Añadir una capa oculta con 16 neuronas
model.add(Dense(16, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(16, activation='relu'))



# Añadir una capa de salida con una sola neurona

model.add(Dense(5, activation='Softmax'))

In [159]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=CategoricalAccuracy())
history = model.fit(features, target, epochs=1000, verbose=0)

In [ ]:
f = plt.figure()
f.set_figwidth(30)
f.set_figheight(9)

plt.plot(history.history['categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('epoch')


<h2 style="text-align:center;">Red Neuronal Sklearn</h2>

In [9]:
clf = MLPClassifier(solver='adam', alpha=1e-5, activation="relu", early_stopping=True,
                    learning_rate="adaptive", hidden_layer_sizes=(3000, 200), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.32142857142857145 
El score en el test fue:  0.2755102040816326


In [10]:
clf = MLPClassifier(solver='adam', alpha=1e-5, activation="relu",
                    hidden_layer_sizes=(3000, 200), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.6275510204081632 
El score en el test fue:  0.25510204081632654


In [11]:
clf = MLPClassifier(solver='adam', alpha=1e-5, activation="relu",
                    hidden_layer_sizes=(30000, 200), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.25510204081632654 
El score en el test fue:  0.14285714285714285


In [12]:
clf = MLPClassifier(solver='adam', alpha=1e-5, activation="relu",
                    hidden_layer_sizes=(50000, 2000), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.6836734693877551 
El score en el test fue:  0.25510204081632654


In [3]:
## Por el momento este es el mejor
clf = MLPClassifier(solver='sgd', alpha=1e-5, activation="relu",
                    hidden_layer_sizes=(30, 20000), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.3431542461005199 
El score en el test fue:  0.2912280701754386


In [7]:
clf = MLPClassifier(solver='sgd', alpha=1e-5, activation="relu",
                    hidden_layer_sizes=(30, 20000), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.43116883116883115 
El score en el test fue:  0.39473684210526316


In [18]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-50, activation="relu",
                    hidden_layer_sizes=(30, 20000), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.5714285714285714 
El score en el test fue:  0.25510204081632654


In [23]:
clf = MLPClassifier(solver='sgd', alpha=1e-50, activation="relu", learning_rate= "adaptive",
                    hidden_layer_sizes=(30000, 200), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.336734693877551 
El score en el test fue:  0.29591836734693877


In [24]:
clf = MLPClassifier(solver='sgd', alpha=1e-50, activation="relu", learning_rate= "adaptive",
                    hidden_layer_sizes=(3000, 200), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.336734693877551 
El score en el test fue:  0.30612244897959184


In [49]:
clf = MLPClassifier(solver='adam', alpha=1e-50, activation="tanh", early_stopping=True, validation_fraction= 0.001,
                    hidden_layer_sizes=(30, 20000), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.30612244897959184 
El score en el test fue:  0.25510204081632654


In [50]:
clf = MLPClassifier(solver='sgd', alpha=1e-50, activation="tanh", early_stopping=True, validation_fraction= 0.001,
                    hidden_layer_sizes=(30, 20000), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.30612244897959184 
El score en el test fue:  0.2755102040816326


In [51]:
clf = MLPClassifier(solver='sgd', activation="relu", learning_rate= "adaptive",
                    hidden_layer_sizes=(30000, 2000), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.336734693877551 
El score en el test fue:  0.30612244897959184


In [24]:
clf = MLPClassifier(solver='sgd', alpha=1e-5, activation="relu",
                    hidden_layer_sizes=(3000, 2000), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.29591836734693877 
El score en el test fue:  0.3469387755102041


<h2 style="text-align:center;">Kmeans</h2>

In [16]:
df_copy = maestro.copy()
df_copy = df_copy[["COLOR", "ESTETICA", "LARGO", "SILUETA", "calif"]]
categorical_columns = df_copy.select_dtypes(include='object').columns
label_encoders = {col: LabelEncoder() for col in categorical_columns}
for col in categorical_columns:
    df_copy[col] = label_encoders[col].fit_transform(df_copy[col])
features = df_copy[["COLOR", "ESTETICA", "LARGO", "SILUETA"]]
features = preprocessing.StandardScaler().fit_transform(features)
target = df_copy["calif"]

In [95]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(features)
y_pred = kmeans.predict(features)
kmeans.score(features, y_pred)

-521.3988842277219

In [17]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(features)
y_pred = kmeans.predict(features)
kmeans.score(features, y_pred)

-1011.5112867810037

<h2 style="text-align:center;">Random forest</h2>

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.3131868131868132


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.3157894736842105


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.2807017543859649


<h2 style="text-align:center;">Regresión logistica</h2>

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.32280701754385965


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.39473684210526316


In [135]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.35714285714285715


<h2 style="text-align:center;">Hierechical clustering</h2>

In [136]:
from sklearn.cluster import AgglomerativeClustering

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
cluster = AgglomerativeClustering(n_clusters=5)
y_pred = cluster.fit_predict(X_train)
y_pred = cluster.fit_predict(X_test)
print(accuracy_score(y_test, y_pred))

0.15934065934065933


In [35]:
from sklearn.cluster import AgglomerativeClustering

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
cluster = AgglomerativeClustering(n_clusters=5)
y_pred = cluster.fit_predict(X_train)
y_pred = cluster.fit_predict(X_test)
print(accuracy_score(y_test, y_pred))

0.14136125654450263


In [6]:
from sklearn.cluster import AgglomerativeClustering

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
cluster = AgglomerativeClustering(n_clusters=5)
y_pred = cluster.fit_predict(X_train)
y_pred = cluster.fit_predict(X_test)
print(accuracy_score(y_test, y_pred))

0.14385964912280702


<h2 style="text-align:center;">Arbol de desicion</h2>

In [48]:
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.29292929292929293


In [113]:
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.2736842105263158


In [7]:
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.27017543859649124


<h2 style="text-align:center;">GaussianNB</h2>

In [138]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)

clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.34615384615384615


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)

clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.4052631578947368


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)

clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.30526315789473685


<h2 style="text-align:center;">SVM (máquinas de vectores de soporte)</h2>

In [38]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.32460732984293195


In [9]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.38421052631578945


In [9]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.3017543859649123


<h2 style="text-align:center;">Gaussian Mixture Model</h2>

In [51]:
from sklearn.mixture import GaussianMixture

clf = GaussianMixture(n_components=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.1717171717171717


In [140]:
from sklearn.mixture import GaussianMixture

clf = GaussianMixture(n_components=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.09340659340659341


In [10]:
from sklearn.mixture import GaussianMixture

clf = GaussianMixture(n_components=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.23859649122807017


<h2 style="text-align:center;">Perceptron</h2>

In [11]:
from sklearn.linear_model import Perceptron

clf = Perceptron()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.16842105263157894


In [11]:
from sklearn.linear_model import Perceptron

clf = Perceptron()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.38421052631578945


In [24]:
from sklearn.linear_model import Perceptron

clf = Perceptron()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.34065934065934067
